# 处理不平衡数据

所谓的不平衡数据集指的是各类的样本量极不平衡。

不平衡数据集的处理方法主要分为两个方面：
1. 从数据角度出发，主要方法为采样，分为欠采样和过采样以及对应的一些改进方法。
2. 从算法的角度出发，考虑不同误分类情况代价的差异性对算法进行优化，主要是基于代价敏感学习算法。代表的算法有Adacost;


## 2. 随机采样

### 2.1 朴素随机过采样（上采样）

针对不平衡数据，最简单的一种方法就是生成少量类的样本，这其中最基本的一种方法就是：从少数类的样本中进行随机采样来增加新的样本，对应的Python库中函数为RandomOverSample:
```
def Ros(data,label):
    from imblearn.over_sampling import RandomOverSampler
    
    ros = RandomOverSampler(random_state = 0)
    data, label = ros.fit_resample(data, label)
    return data, label
```

### 2.2 朴素随机欠采样（下采样）

与过采样相反，欠采样是从多数类样本中随机选择少量样本，再合并原有少数类样本作为新的训练数据集。

随机欠采样有两种类型分别为有放回和无放回两种。

对应Python库中函数为RandomUnderSampler，通过设置RandomUnderSampler中的replacement=True参数, 可以实现自助法(boostrap)抽样。


## 3. 过采样的改进：SMOTE与ADASYN

相对于采样随机的方法进行过采样，还有两种比较流行的过采样的改进方式：

1. Synthetic Minority Oversampling Technique (SMOTE)
2. Adaptive Synthetic (ADASYN)

### 3.1 SMOTE

SMOTE算法的基本思想是对少数类样本进行分析并根据少数类样本人工合成新样本添加到数据集中，具体下图所示，算法流程：

1. 对于少数类中每一个样本$x$，计算该点与少数类中其他样本点的距离，得到最近的k个近邻（即对少数类点进行KNN算法）
2. 根据样本不平衡比例设置一个采样比例以确定采样倍率，对于每一个少数类样本$x$，从其k近邻中随机选择若干个样本，假设选择的近邻为$x^\prime$。
3. 对于每一个随机选出来的近邻$x^\prime$，分别与原样本按照如下的公式构建新的样本：
   $$x_{new}=x+rand(0,1)*(x^\prime - x)$$

但是SMOTE算法缺点也是十分明显：一方面增加了类之间重叠的可能性（由于对每个少数样本都生成新样本，因此容易发生样本重叠）；另一方面是生成一些没有提供有益的样本。

```
from imblearn.over_sampling import SMOTE

X_resampled_smote, y_resampled_smote = SMOTE().fit_resample(X, y)
```
![](./SMOTE.png)

### 3.1.2 SMOTE的改进： Borderline-SMOTE

Borderline-SMOTE与原始SMOTE不同的地方在于，原始的SMOTE是对所有少数类样本生成新样本。而改进的方法则是先根据规则判断出少数类的边界样本，再对这些样本生成新样本。

判断边界的一个简单的规则为：K近邻中有一半以上多数类样本的少数类为边界样本。直观地讲，只为那些周围大部分是多数类样本的少数类样本生成新样本。

假设a为少数类中的一个样本，此时少数类的样本分为三类，如下图所示：

(i) 噪音样本(noise)， 该少数类的所有最近邻样本都来自于不同于样本a的其他类别：

(ii) 危险样本(in danger)， 至少一半的最近邻样本来自于同一类(不同于a的类别)；

(iii) 安全样本(safe)， 所有的最近邻样本都来自于同一个类。

对应的Python库中的实现有三种可以选择的规则：

SMOTE函数中的kind参数控制了选择哪种规则：

borderline1：最近邻中的随机样本与该少数类样本a来自于不同的类；

borderline2：最近邻中的随机样本可以是属于任何一个类的样本；

svm：使用支持向量机分类器产生支持向量然后再生成新的少数类样本。

具体实现如下：
```
from imblearn.under_sampling

import ClusterCentroids

cc = ClusterCentroids(random_state=0)

X_resampled, y_resampled = cc.fit_sample(X, y)
```

![](./SMOTE_2.png)

## 3.2 ADASYN

这种改进方法的主要思想是根据数据分布情况为不同的少数类样本生成不同数量的新样本。首先根据最终的平衡程度设定总共需要生成的新少数类样本数量 ，然后为每个少数类样本x计算分布比例。

对应Python库中函数为ADASYN：
```
from imblearn.over_sampling import ADASYN

X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_sample(X, y)
```

# 4. 欠采样的改进：EasyEnsemble, BalanceCascade, NearMiss

随机欠采样的问题主要是信息丢失，为了解决信息丢失的问题提出了以下几种改进的方式：

1、EasyEnsemble，利用模型融合的方法（Ensemble）：

多次过采样（放回采样，这样产生的训练集才相互独立）产生多个不同的训练集，进而训练多个不同的分类器，通过组合多个分类器的结果得到最终的结果。简单的最佳实践是建立n个模型，每个模型使用少数类的所有样本和多数类的n个不同样本。假设二分类数据集的正负类比例为50000:1000，最后要得到10个模型，那么将保留负类的1000个样本，并随机采样得到10000个正类样本。

然后，将10000个样本成10份，每一份与负类样本组合得到新的子训练集，训练10个不同的模型。

 ```
 EasyEnsemble方法对应Python库中函数为EasyEnsemble，有两个很重要的参数: (i) n_subsets控制的是子集的个数 ；(ii) replacement决定是有放回还是无放回的随机采样。

from imblearn.ensemble

import EasyEnsemble

ee = EasyEnsemble(random_state=0, n_subsets=10)

X_resampled, y_resampled = ee.fit_sample(X, y)
 ```

2、BalanceCascade，利用增量训练的思想（Boosting）：

先通过一次下采样产生训练集，训练一个分类器，对于那些分类正确的多数类样本不放回，然后对这个更小的多数类样本下采样产生训练集，训练第二个分类器，以此类推，最终组合所有分类器的结果得到最终结果。

 ```
 BalanceCascade方法对应Python库中函数为BalanceCascade，有三个很重要的参数: (i) estimator是选择使用的分类器；(ii) n_max_subset控制的是子集的个数；(iii)  bootstrap决定是有放回还是无放回的随机采样。
 
from imblearn.ensemble

import BalanceCascade

from sklearn.linear_model

import LogisticRegression

bc = BalanceCascade(random_state=0,

　　　　　　　　　　estimator=LogisticRegression(random_state=0),

　　　　　　　　　　n_max_subset=4)

X_resampled, y_resampled = bc.fit_sample(X, y)
```

3、NearMiss，利用KNN试图挑选那些最具代表性的多数类样本：

首先计算出每个样本点之间的距离，通过一定规则来选取保留的多数类样本点。因此该方法的计算量通常很大。

```
NearMiss方法对应Python库中函数为NearMiss，通过version来选择使用的规则：

NearMiss-1：选择离N个近邻的负样本的平均距离最小的正样本；

NearMiss-2：选择离N个负样本最远的平均距离最小的正样本；

NearMiss-3：是一个两段式的算法。 首先，对于每一个负样本， 保留它们的M个近邻样本；接着, 那些到N个近邻样本平均距离最大的正样本将被选择。

 

from imblearn.under_sampling

import NearMiss nm1 = NearMiss(random_state=0, version=1)

X_resampled_nm1, y_resampled = nm1.fit_sample(X, y)
```